In [19]:
# Dependencies
import requests
import json
from pprint import pprint
import pandas as pd
import numpy as np
import pymongo

# Import API Key
from config import api_key

In [20]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [21]:
# Define database and collection
db = client.schools_db
collection = db.items

In [23]:
# URL for GET requests to retrieve school data
base_url = "https://api.data.gov/ed/collegescorecard/v1/schools?api_key=" + api_key

In [24]:
# Filter by School "id"
ST = "&school.state=CA"
min_size = "&latest.student.size__range=1000.."
# sort = "&_sort=latest.student.size"


# SZ = "&student.size__range=1000.."
filter_url = base_url + ST + min_size
# filter_url = base_url + ST + min_size + sort
# filter_url = base_url + ST + SZ
# url = base_url + "api_key=" + api_key + "&" + ST + "&" + min_size
# url = base_url + "api_key=" + api_key + "&" + ST + "&" + min_size + "&" + display

In [25]:
print(filter_url)

https://api.data.gov/ed/collegescorecard/v1/schools?api_key=rjhyKFWn0g9PsqjQC1dH9vRABMgQOMzdY5msysWA&school.state=CA&latest.student.size__range=1000..


In [26]:
response = requests.get(filter_url).json()

In [27]:
school_responses = response['metadata']['total']
max_pages = 20
pages = int(np.ceil(school_responses / max_pages))
print(pages)

11


In [28]:
def api_call(page):
    
    school_data = []    
    
    url = filter_url + f"&_page={page}"
    data = requests.get(url).json()
    
    
    for i in range(len(data["results"])):
        s_id = data["results"][i]["id"]

        #SCHOOL DATA
        school_name = data["results"][i]["school"]["name"]
        url = data["results"][i]["school"]["school_url"]
        size = data["results"][i]["latest"]["student"]["size"]

        school_data.append({"ID": s_id,
                            "Name": school_name,
                            "Website": url,
                            "Total Students": size,})
    return school_data

In [42]:
colleges = []

for page_iteration in range(pages):
    college_data = api_call(page_iteration)
    for college in college_data:
        colleges.append(college)
#         print(college)    
# colleges;
college_df = pd.DataFrame(colleges)

In [32]:
# college_df = pd.DataFrame(colleges)

In [43]:
sorted_df = college_df.sort_values("Name")

In [44]:
sorted_df.head()

,ID,Name,Total Students,Website
73,108232,Academy of Art University,7520,www.academyart.edu
41,108807,Allan Hancock College,9282,www.hancockcollege.edu/
203,109040,American Career College-Los Angeles,1237,americancareercollege.edu
3,447768,American Career College-Ontario,1275,americancareercollege.edu
105,109208,American River College,29082,www.arc.losrios.edu/


In [36]:
collection.insert_many(sorted_df.to_dict('records'))
# db.insert_many(sorted_df)

In [37]:
documents = collection.find()

In [38]:
df = pd.DataFrame(list(documents))

In [40]:
df.head()

,ID,Name,Total Students,Website,_id
0,108232,Academy of Art University,7520,www.academyart.edu,5d5c752144d7079dbb0645d9
1,108807,Allan Hancock College,9282,www.hancockcollege.edu/,5d5c752144d7079dbb0645da
2,109040,American Career College-Los Angeles,1237,americancareercollege.edu,5d5c752144d7079dbb0645db
3,447768,American Career College-Ontario,1275,americancareercollege.edu,5d5c752144d7079dbb0645dc
4,109208,American River College,29082,www.arc.losrios.edu/,5d5c752144d7079dbb0645dd
